## Using PyTorch Tensors 

A PyTorch Tensor is basically the same as a numpy multidimensional array: it does not know
anything about deep learning or computational graphs or gradients, and is just
a generic n-dimensional array to be used for arbitrary numeric computation.

The biggest difference between a numpy array and a PyTorch Tensor is that
a PyTorch Tensor can run on either CPU or GPU. To run operations on the GPU,
just cast the Tensor to a cuda datatype.

(This example is adapted from: https://pytorch.org/tutorials/beginner/pytorch_with_examples.html )

### Basic tensor matrix operations

In [1]:
import torch

# Simple matrix
m1 = torch.ones(3, 4)
print('Matrix m1: \n', m1)
print(m1.shape,"\n")

# Another (random) matrix
m2 = torch.rand(3, 4) # fill 3x4 matrix with uniform random numbers in [0,1] interval
print('Matrix m2: \n', m2)
print(m2.shape,"\n")

# Transpose of a matrix
print('Matrix m2^T: \n', m2.t())
print(m2.t().shape,"\n")

# Matrix operations
m3 = m1*m2      # Not a matrix multiplication ! Here each value in matrix m1 are multiplied by value m2
print('Matrix m3: \n', m3)
print(m3.shape,"\n")

# Matrix multiplication using torch.mm
m4 = m1.mm(m2.t())
print('Matrix m4: \n', m4)
print(m4.shape,"\n")

# Matrix multiplication using torch.matmul
m5 = m1.matmul(m2.t())
print('Matrix m5: \n', m5)
print(m5.shape,"\n")

# For matrix-vector multiplication there is also the possibility to use torch.mv
vec = torch.tensor([1.0,2.0,3.0])
print(torch.matmul(m5, vec))

Matrix m1: 
 tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])
torch.Size([3, 4]) 

Matrix m2: 
 tensor([[0.1157, 0.4705, 0.1604, 0.6519],
        [0.5901, 0.3330, 0.9829, 0.5998],
        [0.5548, 0.0507, 0.7762, 0.8113]])
torch.Size([3, 4]) 

Matrix m2^T: 
 tensor([[0.1157, 0.5901, 0.5548],
        [0.4705, 0.3330, 0.0507],
        [0.1604, 0.9829, 0.7762],
        [0.6519, 0.5998, 0.8113]])
torch.Size([4, 3]) 

Matrix m3: 
 tensor([[0.1157, 0.4705, 0.1604, 0.6519],
        [0.5901, 0.3330, 0.9829, 0.5998],
        [0.5548, 0.0507, 0.7762, 0.8113]])
torch.Size([3, 4]) 

Matrix m4: 
 tensor([[1.3984, 2.5058, 2.1929],
        [1.3984, 2.5058, 2.1929],
        [1.3984, 2.5058, 2.1929]])
torch.Size([3, 3]) 

Matrix m5: 
 tensor([[1.3984, 2.5058, 2.1929],
        [1.3984, 2.5058, 2.1929],
        [1.3984, 2.5058, 2.1929]])
torch.Size([3, 3]) 

tensor([12.9888, 12.9888, 12.9888])


### More operations (power, sum, clamp, ...)

(See even more basic operations here: https://jhui.github.io/2018/02/09/PyTorch-Basic-operations/)

In [2]:
mat = torch.rand(3, 4)
print(mat)

# Takes the power of each element in input
print(mat.pow(3))

# Returns the sum of all elements in the input tensor
print(mat.sum())

# Clamp all elements in input into the range [ min, max ] and return a resulting tensor
print(mat.clamp(0.3,0.7))

tensor([[0.3053, 0.3329, 0.1608, 0.3585],
        [0.2057, 0.0540, 0.3088, 0.3913],
        [0.1058, 0.3738, 0.0864, 0.2156]])
tensor([[0.0285, 0.0369, 0.0042, 0.0461],
        [0.0087, 0.0002, 0.0294, 0.0599],
        [0.0012, 0.0522, 0.0006, 0.0100]])
tensor(2.8989)
tensor([[0.3053, 0.3329, 0.3000, 0.3585],
        [0.3000, 0.3000, 0.3088, 0.3913],
        [0.3000, 0.3738, 0.3000, 0.3000]])


### Convertion to numpy array

In [3]:
import numpy as np

# convertion PyTorch -> numpy
a = torch.randn(5)
b = a.numpy()

# convertion numpy -> PyTorch
c = torch.from_numpy(b)

print(a)
print(b)
print(c)

tensor([-1.7202,  0.8410, -0.3200,  1.4264,  1.1526])
[-1.7201914   0.8410047  -0.32003662  1.42642     1.1526212 ]
tensor([-1.7202,  0.8410, -0.3200,  1.4264,  1.1526])


### Autograd

A PyTorch Tensor represents a node in a computational graph. If ``x`` is a
Tensor that has ``x.requires_grad=True`` then ``x.grad`` is another Tensor
holding the gradient of ``x`` with respect to some scalar value.

Example for simple regression (adapted from S. Sviridov lecture):

In [4]:
# Simple regression example
x = [1., 2., 3., 4., 5.]           # data
y = [10., 20., 30., 40., 50.]      # target values

# Gradients will be calculated w.r.t this tensors (has "requires_grad=True")
w = torch.tensor([1.],requires_grad=True)

# Number of loops on all sample
for epoch in range(5):
    
    # Loop on data events and target values
    for x_i, y_i in zip(x, y):
        
        # compute predicted target variable
        y_pred = x_i * w
                
        # compute Mean Squared Error (MSE)
        loss = (y_pred - y_i) ** 2
        
        # With PyTorch we can automatically compute the derivative of the loss 
        # w.r.t. the tensors that have requires_grad set to True (i.e. weights).
        # compute gradients
        loss.backward()
                        
        print('\t x=%.1f y=%.1f, w=%.2f, dloss/dw=%.1f' % (x_i, y_i, w.data, w.grad.data))
                
        # make one step towards the local minima, with learning rate 0.01
        w.data -= 0.01 * w.grad.data
        
        # clear gradients after updating weights
        w.grad.data.zero_()
        
    print('Loss at epoch #%d: %.6f \n' % (epoch+1, loss.data[0]))

print('Final: w = %.4f' % (w.data))


	 x=1.0 y=10.0, w=1.00, dloss/dw=-18.0
	 x=2.0 y=20.0, w=1.18, dloss/dw=-70.6
	 x=3.0 y=30.0, w=1.89, dloss/dw=-146.1
	 x=4.0 y=40.0, w=3.35, dloss/dw=-212.9
	 x=5.0 y=50.0, w=5.48, dloss/dw=-226.2
Loss at epoch #1: 511.797760 

	 x=1.0 y=10.0, w=7.74, dloss/dw=-4.5
	 x=2.0 y=20.0, w=7.78, dloss/dw=-17.7
	 x=3.0 y=30.0, w=7.96, dloss/dw=-36.7
	 x=4.0 y=40.0, w=8.33, dloss/dw=-53.5
	 x=5.0 y=50.0, w=8.86, dloss/dw=-56.9
Loss at epoch #2: 32.337894 

	 x=1.0 y=10.0, w=9.43, dloss/dw=-1.1
	 x=2.0 y=20.0, w=9.44, dloss/dw=-4.5
	 x=3.0 y=30.0, w=9.49, dloss/dw=-9.2
	 x=4.0 y=40.0, w=9.58, dloss/dw=-13.5
	 x=5.0 y=50.0, w=9.71, dloss/dw=-14.3
Loss at epoch #3: 2.043265 

	 x=1.0 y=10.0, w=9.86, dloss/dw=-0.3
	 x=2.0 y=20.0, w=9.86, dloss/dw=-1.1
	 x=3.0 y=30.0, w=9.87, dloss/dw=-2.3
	 x=4.0 y=40.0, w=9.89, dloss/dw=-3.4
	 x=5.0 y=50.0, w=9.93, dloss/dw=-3.6
Loss at epoch #4: 0.129104 

	 x=1.0 y=10.0, w=9.96, dloss/dw=-0.1
	 x=2.0 y=20.0, w=9.96, dloss/dw=-0.3
	 x=3.0 y=30.0, w=9.97, dloss/d

### Exercice
Now we have the following data. How should be modified the above code ? How many epochs are needed for convergence ?


In [6]:
# Now we have the following data. How should be modified the above code 
x = [1., 2., 3., 4., 5., 6., 7., 8., 9., 10.]           # data
y = [15., 25., 35., 45., 55., 65., 75., 85., 95., 105.] # target values

# Gradients will be calculated w.r.t these two tensors (have "requires_grad=True")
w = torch.tensor([1.],requires_grad=True)
b = torch.tensor([0.],requires_grad=True)

for epoch in range(100):
    
    for x_i, y_i in zip(x, y):
        
        # compute predicted target variable
        y_pred = x_i * w + b
                
        # compute Mean Squared Error (MSE)
        loss = (y_pred - y_i) ** 2
        
        # With PyTorch we can automatically compute the derivative of the loss 
        # w.r.t. the tensors that have requires_grad set to True (i.e. w and b).
        # compute gradients
        loss.backward()
                        
        print('\t x=%.1f y=%.1f, w=%.2f, b=%.2f, dloss/dw=%.1f, dloss/db=%.1f' % (x_i, y_i, w.data, b.data, w.grad.data, b.grad.data))
                
        # make one step towards the local minima, with learning rate 0.01
        w.data -= 0.01 * w.grad.data
        b.data -= 0.01 * b.grad.data
        
        # clear gradients after updating weights
        w.grad.data.zero_()
        b.grad.data.zero_()
        
    print('Loss at epoch #%d: %.6f \n' % (epoch+1, loss.data[0]))

print('Final: w = %.4f, b = %4f' % (w.data, b.data))


	 x=1.0 y=15.0, w=1.00, b=0.00, dloss/dw=-28.0, dloss/db=-28.0
	 x=2.0 y=25.0, w=1.28, b=0.28, dloss/dw=-88.6, dloss/db=-44.3
	 x=3.0 y=35.0, w=2.17, b=0.72, dloss/dw=-166.7, dloss/db=-55.6
	 x=4.0 y=45.0, w=3.83, b=1.28, dloss/dw=-227.1, dloss/db=-56.8
	 x=5.0 y=55.0, w=6.10, b=1.85, dloss/dw=-226.3, dloss/db=-45.3
	 x=6.0 y=65.0, w=8.37, b=2.30, dloss/dw=-150.0, dloss/db=-25.0
	 x=7.0 y=75.0, w=9.87, b=2.55, dloss/dw=-47.3, dloss/db=-6.8
	 x=8.0 y=85.0, w=10.34, b=2.62, dloss/dw=5.4, dloss/db=0.7
	 x=9.0 y=95.0, w=10.29, b=2.61, dloss/dw=3.3, dloss/db=0.4
	 x=10.0 y=105.0, w=10.25, b=2.61, dloss/dw=2.7, dloss/db=0.3
Loss at epoch #1: 0.018289 

	 x=1.0 y=15.0, w=10.23, b=2.60, dloss/dw=-4.3, dloss/db=-4.3
	 x=2.0 y=25.0, w=10.27, b=2.65, dloss/dw=-7.3, dloss/db=-3.6
	 x=3.0 y=35.0, w=10.34, b=2.68, dloss/dw=-7.7, dloss/db=-2.6
	 x=4.0 y=45.0, w=10.42, b=2.71, dloss/dw=-4.9, dloss/db=-1.2
	 x=5.0 y=55.0, w=10.47, b=2.72, dloss/dw=0.6, dloss/db=0.1
	 x=6.0 y=65.0, w=10.46, b=2.72, dlos

	 x=5.0 y=55.0, w=10.03, b=4.87, dloss/dw=0.0, dloss/db=0.0
	 x=6.0 y=65.0, w=10.03, b=4.87, dloss/dw=0.3, dloss/db=0.1
	 x=7.0 y=75.0, w=10.02, b=4.87, dloss/dw=0.4, dloss/db=0.1
	 x=8.0 y=85.0, w=10.02, b=4.87, dloss/dw=0.3, dloss/db=0.0
	 x=9.0 y=95.0, w=10.02, b=4.87, dloss/dw=0.2, dloss/db=0.0
	 x=10.0 y=105.0, w=10.01, b=4.87, dloss/dw=0.1, dloss/db=0.0
Loss at epoch #83: 0.000055 

	 x=1.0 y=15.0, w=10.01, b=4.87, dloss/dw=-0.2, dloss/db=-0.2
	 x=2.0 y=25.0, w=10.01, b=4.87, dloss/dw=-0.4, dloss/db=-0.2
	 x=3.0 y=35.0, w=10.02, b=4.87, dloss/dw=-0.4, dloss/db=-0.1
	 x=4.0 y=45.0, w=10.02, b=4.87, dloss/dw=-0.3, dloss/db=-0.1
	 x=5.0 y=55.0, w=10.03, b=4.87, dloss/dw=0.0, dloss/db=0.0
	 x=6.0 y=65.0, w=10.03, b=4.87, dloss/dw=0.3, dloss/db=0.1
	 x=7.0 y=75.0, w=10.02, b=4.87, dloss/dw=0.4, dloss/db=0.1
	 x=8.0 y=85.0, w=10.02, b=4.87, dloss/dw=0.3, dloss/db=0.0
	 x=9.0 y=95.0, w=10.02, b=4.87, dloss/dw=0.2, dloss/db=0.0
	 x=10.0 y=105.0, w=10.01, b=4.87, dloss/dw=0.1, dloss/db=0.


NN with tensor and autograd
-------------------------------

We consider a fully-connected ReLU network with one hidden layer and no biases, trained to predict y from x using Euclidean error.

The model that we want to build has the following structure:
$$\hat{y}(x) = \text{relu}(x.w_1).w_2,$$
where $x$ and $y$ are the input and output features (of dimension 1000 and 10, respectively). Here the relu activation function is used and $w_1$ and $w_2$ are weight matrices.

The network consists of a fully-connected ReLU network with one hidden layer and no biases, trained to predict y from x by minimizing squared Euclidean distance.

This implementation computes the forward pass using operations on PyTorch Tensors, and uses PyTorch autograd to compute gradients.


In [1]:
%matplotlib inline

import torch

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold input and outputs.
# Setting requires_grad=False indicates that we do not need to compute gradients
# with respect to these Tensors during the backward pass.
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Create random Tensors for weights.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y using operations on Tensors; these
    # are exactly the same operations we used to compute the forward pass using
    # Tensors, but we do not need to keep references to intermediate values since
    # we are not implementing the backward pass by hand.
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    
    # alternativelt you can use the built-in pytorch relu function
    # y_pred = torch.relu(x.mm(w1)).mm(w2)

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the a scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    print(t, loss.item())

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call w1.grad and w2.grad will be Tensors holding the gradient
    # of the loss with respect to w1 and w2 respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    # An alternative way is to operate on weight.data and weight.grad.data.
    # Recall that tensor.data gives a tensor that shares the storage with
    # tensor, but doesn't track history.
    # You can also use torch.optim.SGD to achieve this.
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # Manually zero the gradients after updating weights
        w1.grad.zero_()
        w2.grad.zero_()

0 30678590.0
1 25521168.0
2 23136452.0
3 20305556.0
4 16234543.0
5 11643255.0
6 7652273.0
7 4800460.5
8 3007879.75
9 1949954.25
10 1333952.5
11 967682.0625
12 739503.875
13 588779.375
14 482949.90625
15 404452.625
16 343662.75
17 294943.21875
18 255164.59375
19 222171.734375
20 194419.09375
21 170843.21875
22 150686.75
23 133354.125
24 118375.625
25 105375.421875
26 94043.390625
27 84146.6640625
28 75488.6015625
29 67864.171875
30 61132.984375
31 55179.12890625
32 49901.2109375
33 45205.5625
34 41021.734375
35 37282.4375
36 33935.19140625
37 30932.11328125
38 28231.67578125
39 25799.919921875
40 23605.1953125
41 21621.150390625
42 19825.15625
43 18196.712890625
44 16718.09375
45 15373.5673828125
46 14150.146484375
47 13035.361328125
48 12018.01171875
49 11088.7197265625
50 10239.16015625
51 9461.2841796875
52 8748.724609375
53 8094.9970703125
54 7495.0166015625
55 6943.86279296875
56 6437.087890625
57 5970.7197265625
58 5541.384765625
59 5145.693359375
60 4780.80322265625
61 4443.99609

453 8.810096187517047e-05
454 8.656635327497497e-05
455 8.475106005789712e-05
456 8.352175063919276e-05
457 8.235369750764221e-05
458 8.125367457978427e-05
459 7.997159264050424e-05
460 7.887403626227751e-05
461 7.721551082795486e-05
462 7.62003764975816e-05
463 7.511214062105864e-05
464 7.390701648546383e-05
465 7.257351535372436e-05
466 7.151535101002082e-05
467 7.078088674461469e-05
468 6.999324250500649e-05
469 6.884289177833125e-05
470 6.808150646975264e-05
471 6.700786616420373e-05
472 6.618237239308655e-05
473 6.498227594420314e-05
474 6.421494617825374e-05
475 6.33703893981874e-05
476 6.239462527446449e-05
477 6.163326179375872e-05
478 6.086544453864917e-05
479 6.017292616888881e-05
480 5.8904501202050596e-05
481 5.798885831609368e-05
482 5.715732186217792e-05
483 5.650144521496259e-05
484 5.598360439762473e-05
485 5.509749462362379e-05
486 5.459368185256608e-05
487 5.377618072088808e-05
488 5.293567664921284e-05
489 5.210505332797766e-05
490 5.152084122528322e-05
491 5.1013219

### Optional: Exercice
Add one more hidden layer with 100 neurons. Use successively sigmoid (torch.sigmoid) and relu (torch.relu) activation functions.

Note: you might need to increase the numbers of epochs (this solution is much less effective than the previous one !)


In [1]:
import numpy as np

%matplotlib inline

import torch

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H1, H2, D_out = 64, 1000, 100, 100, 10

# Create random Tensors to hold input and outputs.
# Setting requires_grad=False indicates that we do not need to compute gradients
# with respect to these Tensors during the backward pass.
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Create random Tensors for weights.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
w1 = torch.randn(D_in, H1, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H1, H2, device=device, dtype=dtype, requires_grad=True)
w3 = torch.randn(H2, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(10000):
    
    # y_pred with sigmoid activation function
    y_pred = torch.relu(torch.sigmoid(x.mm(w1)).mm(w2)).mm(w3)
    
    #y_pred = x.mm(w1).clamp(min=0).mm(w2).clamp(min=0).mm(w3)
        
    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the a scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    print(t, loss.item())
    
    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call w1.grad and w2.grad will be Tensors holding the gradient
    # of the loss with respect to w1 and w2 respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    # An alternative way is to operate on weight.data and weight.grad.data.
    # Recall that tensor.data gives a tensor that shares the storage with
    # tensor, but doesn't track history.
    # You can also use torch.optim.SGD to achieve this.
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        w3 -= learning_rate * w3.grad

        # Manually zero the gradients after updating weights
        w1.grad.zero_()
        w2.grad.zero_()
        w3.grad.zero_()

0 3103784.5
1 1282255.625
2 795089.5625
3 600897.1875
4 509082.4375
5 457848.9375
6 425294.40625
7 402190.46875
8 384108.71875
9 368774.34375
10 355053.03125
11 342778.40625
12 331371.5
13 320519.875
14 310609.75
15 301462.21875
16 292959.3125
17 284956.4375
18 277341.1875
19 270160.40625
20 263362.1875
21 256870.015625
22 250618.828125
23 244623.625
24 238913.859375
25 233407.46875
26 228145.390625
27 223097.171875
28 218154.09375
29 213378.78125
30 208779.90625
31 204435.71875
32 200293.8125
33 196321.265625
34 192465.046875
35 188824.625
36 185370.296875
37 182055.71875
38 178867.84375
39 175770.703125
40 172789.109375
41 169894.90625
42 167076.640625
43 164338.234375
44 161658.421875
45 159053.3125
46 156527.640625
47 154051.46875
48 151649.015625
49 149293.359375
50 147002.984375
51 144749.6875
52 142531.53125
53 140352.796875
54 138211.953125
55 136085.625
56 133968.578125
57 131877.375
58 129793.5078125
59 127764.8359375
60 125798.921875
61 123894.546875
62 122050.0625
63 120305

638 5187.11181640625
639 5171.03369140625
640 5155.255859375
641 5139.14404296875
642 5123.572265625
643 5107.58154296875
644 5092.03173828125
645 5076.30615234375
646 5060.845703125
647 5045.296875
648 5029.81982421875
649 5014.46533203125
650 4999.2197265625
651 4983.7880859375
652 4968.89013671875
653 4953.5478515625
654 4938.52783203125
655 4923.47802734375
656 4908.52734375
657 4893.6201171875
658 4878.818359375
659 4864.06103515625
660 4849.37158203125
661 4834.6083984375
662 4819.6943359375
663 4804.8662109375
664 4790.068359375
665 4775.44921875
666 4760.75048828125
667 4746.21484375
668 4731.6982421875
669 4717.3896484375
670 4702.91015625
671 4688.744140625
672 4674.44189453125
673 4660.4404296875
674 4646.1259765625
675 4632.23388671875
676 4618.216796875
677 4604.3212890625
678 4590.4482421875
679 4576.6484375
680 4562.91259765625
681 4549.1689453125
682 4535.6435546875
683 4522.01171875
684 4508.5234375
685 4495.02490234375
686 4481.54248046875
687 4468.25146484375
688 445

1061 1644.2021484375
1062 1640.3402099609375
1063 1636.4898681640625
1064 1632.63134765625
1065 1628.7388916015625
1066 1624.88427734375
1067 1621.1273193359375
1068 1617.2794189453125
1069 1613.4619140625
1070 1609.631103515625
1071 1605.93896484375
1072 1602.094482421875
1073 1598.3297119140625
1074 1594.603759765625
1075 1590.883544921875
1076 1587.1160888671875
1077 1583.3876953125
1078 1579.696044921875
1079 1576.0224609375
1080 1572.2933349609375
1081 1568.5989990234375
1082 1564.9766845703125
1083 1561.3201904296875
1084 1557.6383056640625
1085 1553.984375
1086 1550.45556640625
1087 1546.789794921875
1088 1543.1927490234375
1089 1539.5450439453125
1090 1536.0784912109375
1091 1532.421630859375
1092 1528.880859375
1093 1525.303955078125
1094 1521.8017578125
1095 1518.3021240234375
1096 1514.697509765625
1097 1511.24365234375
1098 1507.7840576171875
1099 1504.2601318359375
1100 1500.74853515625
1101 1497.319091796875
1102 1493.8533935546875
1103 1490.3697509765625
1104 1486.901977

1438 717.19287109375
1439 715.7457275390625
1440 714.2810668945312
1441 712.8095703125
1442 711.35986328125
1443 709.897216796875
1444 708.4526977539062
1445 707.0237426757812
1446 705.572509765625
1447 704.1363525390625
1448 702.6922607421875
1449 701.2637939453125
1450 699.8358764648438
1451 698.42431640625
1452 696.9984130859375
1453 695.5755615234375
1454 694.1616821289062
1455 692.751953125
1456 691.3446044921875
1457 689.9575805664062
1458 688.5438842773438
1459 687.15869140625
1460 685.7431640625
1461 684.3617553710938
1462 682.9879760742188
1463 681.6040649414062
1464 680.2140502929688
1465 678.8262329101562
1466 677.4669799804688
1467 676.0897216796875
1468 674.7440795898438
1469 673.3736572265625
1470 671.9944458007812
1471 670.64306640625
1472 669.2832641601562
1473 667.9420166015625
1474 666.5974731445312
1475 665.2467041015625
1476 663.8988647460938
1477 662.565185546875
1478 661.2214965820312
1479 659.9158935546875
1480 658.5682983398438
1481 657.23779296875
1482 655.9227

1916 288.25030517578125
1917 287.7393493652344
1918 287.22723388671875
1919 286.7073669433594
1920 286.210693359375
1921 285.68658447265625
1922 285.18316650390625
1923 284.67913818359375
1924 284.1690979003906
1925 283.665771484375
1926 283.1540832519531
1927 282.6585693359375
1928 282.158447265625
1929 281.64892578125
1930 281.14794921875
1931 280.6592712402344
1932 280.1517639160156
1933 279.6514587402344
1934 279.16473388671875
1935 278.6636962890625
1936 278.17041015625
1937 277.6804504394531
1938 277.18505859375
1939 276.6976013183594
1940 276.19647216796875
1941 275.71441650390625
1942 275.23101806640625
1943 274.73187255859375
1944 274.2555847167969
1945 273.7650146484375
1946 273.28082275390625
1947 272.79949951171875
1948 272.3193664550781
1949 271.8348083496094
1950 271.35455322265625
1951 270.8786315917969
1952 270.39898681640625
1953 269.9221496582031
1954 269.4428405761719
1955 268.976318359375
1956 268.493408203125
1957 268.021240234375
1958 267.555419921875
1959 267.077

...

8182 0.07121844589710236
8183 0.07114005088806152
8184 0.07106531411409378
8185 0.07098536193370819
8186 0.0709107369184494
8187 0.07083368301391602
8188 0.0707566887140274
8189 0.07067746669054031
8190 0.0706048235297203
8191 0.07052437961101532
8192 0.07044880092144012
8193 0.07037371397018433
8194 0.07029485702514648
8195 0.07022039592266083
8196 0.07014574855566025
8197 0.0700707659125328
8198 0.06999540328979492
8199 0.0699169784784317
8200 0.0698390007019043
8201 0.06976521760225296
8202 0.06969375163316727
8203 0.06961347907781601
8204 0.06953755766153336
8205 0.06946417689323425
8206 0.06938939541578293
8207 0.069313183426857
8208 0.06923729926347733
8209 0.06916363537311554
8210 0.06908781826496124
8211 0.06901498883962631
8212 0.06893948465585709
8213 0.06886560469865799
8214 0.06879132241010666
8215 0.06871622800827026
8216 0.06863968074321747
8217 0.068568155169487
8218 0.06849092245101929
8219 0.06841843575239182
8220 0.06834612786769867
8221 0.06827052682638168
8222 0.068

8589 0.04614526778459549
8590 0.04609934240579605
8591 0.04605397209525108
8592 0.046004459261894226
8593 0.045952849090099335
8594 0.04590659588575363
8595 0.04585668072104454
8596 0.04580863565206528
8597 0.04576212540268898
8598 0.04571431875228882
8599 0.045667968690395355
8600 0.045619428157806396
8601 0.045570239424705505
8602 0.0455215722322464
8603 0.04547501727938652
8604 0.04542740434408188
8605 0.04537878930568695
8606 0.04533389210700989
8607 0.045286186039447784
8608 0.04523993656039238
8609 0.04519122838973999
8610 0.045145124197006226
8611 0.045096490532159805
8612 0.04504779726266861
8613 0.04500235617160797
8614 0.044955380260944366
8615 0.04490980505943298
8616 0.04486345872282982
8617 0.044814545661211014
8618 0.04476802796125412
8619 0.04472152143716812
8620 0.04467425495386124
8621 0.04462742805480957
8622 0.04458393156528473
8623 0.04453752189874649
8624 0.04448919743299484
8625 0.044444262981414795
8626 0.044396284967660904
8627 0.04435001313686371
8628 0.0443045

9012 0.029927702620625496
9013 0.029899626970291138
9014 0.02986992709338665
9015 0.02983923628926277
9016 0.029810706153512
9017 0.029779158532619476
9018 0.02975134551525116
9019 0.029720213264226913
9020 0.029690828174352646
9021 0.029661383479833603
9022 0.02963005006313324
9023 0.029600638896226883
9024 0.029570484533905983
9025 0.029542867094278336
9026 0.02951326034963131
9027 0.029482798650860786
9028 0.029455173760652542
9029 0.029426272958517075
9030 0.02939615398645401
9031 0.029367784038186073
9032 0.02933827042579651
9033 0.0293082557618618
9034 0.029277440160512924
9035 0.029248394072055817
9036 0.02921704575419426
9037 0.02918810397386551
9038 0.029161209240555763
9039 0.0291342344135046
9040 0.029103804379701614
9041 0.02907562255859375
9042 0.029046304523944855
9043 0.029016930609941483
9044 0.02898760512471199
9045 0.028958970680832863
9046 0.02892884984612465
9047 0.028902152553200722
9048 0.02887408435344696
9049 0.028846271336078644
9050 0.02881479822099209
9051 0.

9411 0.020251937210559845
9412 0.020231662318110466
9413 0.020212562754750252
9414 0.020193880423903465
9415 0.02017504721879959
9416 0.02015496790409088
9417 0.020135633647441864
9418 0.020116275176405907
9419 0.02009761519730091
9420 0.020076824352145195
9421 0.020059291273355484
9422 0.020038608461618423
9423 0.0200202539563179
9424 0.020000871270895004
9425 0.01998116634786129
9426 0.019962221384048462
9427 0.01994330808520317
9428 0.019923396408557892
9429 0.0199048463255167
9430 0.019886408001184464
9431 0.019868381321430206
9432 0.01984862983226776
9433 0.01983070932328701
9434 0.01981043815612793
9435 0.0197922233492136
9436 0.019772225990891457
9437 0.019754886627197266
9438 0.019735615700483322
9439 0.01971639320254326
9440 0.019696518778800964
9441 0.01967882551252842
9442 0.019660338759422302
9443 0.01963954046368599
9444 0.01962154358625412
9445 0.01960442215204239
9446 0.019586432725191116
9447 0.01956719160079956
9448 0.019547978416085243
9449 0.01953059993684292
9450 0.

9764 0.01455521397292614
9765 0.01454242505133152
9766 0.014527970924973488
9767 0.014515680260956287
9768 0.014503373764455318
9769 0.014489672146737576
9770 0.014476094394922256
9771 0.014462975785136223
9772 0.0144497100263834
9773 0.014436028897762299
9774 0.01442328467965126
9775 0.01440993882715702
9776 0.014396872371435165
9777 0.014383463189005852
9778 0.0143702682107687
9779 0.014357062987983227
9780 0.01434399839490652
9781 0.014331532642245293
9782 0.01431841216981411
9783 0.014306195080280304
9784 0.01429351232945919
9785 0.014280333183705807
9786 0.014266042038798332
9787 0.014253772795200348
9788 0.014239704236388206
9789 0.01422862894833088
9790 0.014214740134775639
9791 0.014201907441020012
9792 0.014188881032168865
9793 0.01417590957134962
9794 0.014162993989884853
9795 0.01415092870593071
9796 0.014137570746243
9797 0.014124948531389236
9798 0.01411128044128418
9799 0.014098618179559708
9800 0.014086735434830189
9801 0.014072826132178307
9802 0.014060791581869125
9803